In [11]:
import pandas as pd #Its used to load, clean and analyze data in table format
import psycopg2 #Its a driver that is used to connect to postgresql
from sqlalchemy import create_engine
import os #Its access the system variables and file path
from dotenv import load_dotenv #It load the scerats from the .env file 

load_dotenv()

DB_HOST = os.getenv("DB_HOST")   #os.getenv() iske ander jo value hoti hai vo ye system se envse nikal ke lata hai specifically 
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_PORT = os.getenv("DB_PORT")

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}") 

"""
postgresql://	     Database type
{DB_USER}:{DB_PASS}	 Username & Password
@{DB_HOST}	         Database server location
:{DB_PORT}	         Database port
/{DB_NAME}	         Specific database inside server

"""

df = pd.read_sql("SELECT * FROM weather_readings ORDER BY timestamp", engine)
df.head()


,id,city,timestamp,temperature,humidity,wind_speed,pm10,pm2_5,nitrogen_dioxide,ozone
0,1,Delhi,2025-11-06 10:00:03.516669,NaN,NaN,NaN,266.4,77.8,13.4,70.0
1,2,Hyderabad,2025-11-06 10:00:03.707155,NaN,NaN,NaN,39.2,38.3,18.3,44.0
2,3,Kolkata,2025-11-06 10:00:03.931717,NaN,NaN,NaN,133.2,125.3,22.3,52.0
3,4,Jaipur,2025-11-06 10:00:04.723255,16.4,65.0,4.3,61.7,43.4,12.3,69.0
4,5,Lucknow,2025-11-06 10:00:05.387228,19.6,91.0,4.9,78.9,67.1,13.7,52.0


In [12]:
df.describe()

,id,timestamp,temperature,humidity,wind_speed,pm10,pm2_5,nitrogen_dioxide,ozone
count,434.000000,434,382.000000,382.000000,382.000000,431.000000,431.000000,431.000000,431.000000
mean,217.500000,2025-11-06 16:30:08.819577600,18.432199,84.714660,5.178534,42.208817,32.503248,14.757541,54.241299
min,1.000000,2025-11-06 10:00:03.516669,-8.400000,24.000000,1.000000,3.600000,3.600000,0.100000,6.000000
25%,109.250000,2025-11-06 13:00:09.136205056,14.700000,81.000000,2.900000,13.050000,11.350000,6.100000,37.000000
50%,217.500000,2025-11-06 16:30:08.483345920,19.700000,84.500000,4.200000,23.700000,21.400000,12.200000,53.000000
75%,325.750000,2025-11-06 20:00:08.500833536,24.400000,93.000000,5.600000,59.400000,49.400000,19.300000,69.000000
max,434.000000,2025-11-06 23:00:12.793834,28.000000,100.000000,23.600000,284.100000,125.300000,85.800000,151.000000
std,125.429263,NaN,7.740183,11.775224,4.383638,47.928622,26.865222,12.476040,23.376638


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                434 non-null    int64         
 1   city              434 non-null    object        
 2   timestamp         434 non-null    datetime64[ns]
 3   temperature       382 non-null    float64       
 4   humidity          382 non-null    float64       
 5   wind_speed        382 non-null    float64       
 6   pm10              431 non-null    float64       
 7   pm2_5             431 non-null    float64       
 8   nitrogen_dioxide  431 non-null    float64       
 9   ozone             431 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 34.0+ KB


In [14]:
df.isnull().sum()

id                   0
city                 0
timestamp            0
temperature         52
humidity            52
wind_speed          52
pm10                 3
pm2_5                3
nitrogen_dioxide     3
ozone                3
dtype: int64

In [15]:
df.nunique()

id                  434
city                 31
timestamp           434
temperature          53
humidity             29
wind_speed           44
pm10                 84
pm2_5                83
nitrogen_dioxide     79
ozone                62
dtype: int64

In [16]:
pd.read_sql("""
SELECT city , MIN(pm10) AS min_pm10
FROM weather_readings
GROUP BY city
ORDER BY min_pm10
LIMIT 10
""", engine)


,city,min_pm10
0,Kohima,3.6
1,Shillong,4.7
2,Gangtok,6.4
3,Leh,7.5
4,Imphal,7.8
5,Kavaratti,9.1
6,Dispur,9.5
7,Chennai,10.3
8,Thiruvananthapuram,12.2
9,Puducherry,14.4


In [18]:
pd.read_sql("""
SELECT
    city,
    ROUND(AVG(temperature)::numeric) AS avg_temp
FROM weather_readings
GROUP BY city
ORDER BY avg_temp DESC""", engine)


,city,avg_temp
0,Thiruvananthapuram,27.0
1,Port Blair,27.0
2,Mumbai,26.0
3,Chennai,26.0
4,Kavaratti,26.0
5,Dispur,25.0
6,Puducherry,25.0
7,Agartala,25.0
8,Panaji,24.0
9,Gandhinagar,22.0
